# Process Data

## Load Data

In [1]:
import xarray as xr

In [2]:
# 2019: download https://storage.googleapis.com/contrails-301217-public-data/2019-erf.orig.nc
# 2024: download https://storage.googleapis.com/contrails-301217-public-data/2014-erf.orig.nc

year = 2024

# load data
ds = xr.open_dataset(f"{year}.nc")

## Image Processing

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colormaps
from matplotlib.colors import LogNorm

In [4]:

# transpose dims
da = ds["ef_net_overlap"].T

# reverse latitude
da = da.isel(latitude=slice(None, None, -1))

cmap = "hot_r"

# Save as PNG with proper extent
plt.imsave(f"{year}.png", np.log10(da.values), cmap=cmap, vmin=-5, vmax=-2) 

## FIRS

In [5]:
import json
import topojson as tp
import shapely
import pandas as pd

In [6]:
# https://static.observableusercontent.com/files/f35567ec50da95d5b9fcbaddb3fc4d7a405a426c04a7888d086a6fc86fe92a0ba299571fbf57332c0689cc24a62ee1884cb637ef51883a1a9c0d0460d5beedec?response-content-disposition=attachment%3Bfilename*%3DUTF-8%27%27worldfirs.json
rpath = "worldfirs.topojson"
with open(rpath, "r") as f:
    firs_topojson = json.load(f)

In [9]:
# load TopoJSON
firs_topo = tp.Topology(firs_topojson)
firs_geojson = json.loads(firs_topo.to_geojson())

In [18]:
# init dataframe
df = pd.DataFrame(f["properties"] for f in firs_geojson["features"]) \
    .drop_duplicates(subset="designator") \
    .set_index("designator")

df["value"] = 0

In [20]:
# sum up EF % within each FIR
for feature in firs_geojson["features"]:
    d = feature["properties"]["designator"]
    if not d:
        continue

    geom = shapely.from_geojson(json.dumps(feature["geometry"]))
    points = shapely.points(ds["longitude"].values[:, None], ds["latitude"].values[None, :])
    mask = shapely.contains(geom, points)

    df.loc[d, "value"] = ds["ef_net_overlap"].where(mask).sum().item()

/var/folders/73/sk8cn3514c7179st_tv17j300000gn/T/ipykernel_62281/1084659805.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.0033245765110668365' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[d, "value"] = ds["ef_net_overlap"].where(mask).sum().item()


In [21]:
# backup file
df.reset_index().to_csv("firs.csv", index=False)

In [22]:
firs = json.loads(firs_topo.to_json())

for geom in firs["objects"]["data"]["geometries"]:
    d = geom["properties"]["designator"]
    if not d:
        continue

    geom["properties"]["value"] = float(df.loc[d, "value"])

In [23]:
with open("firs.topojson", "w") as f:
    json.dump(firs, f, separators=(',', ':'))